<a href="https://colab.research.google.com/github/lartmann/deepfake_detection_project/blob/main/MediaPipe/NVB_Landmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
#from tensorflow_docs.vis import embed
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import cv2
import os

# Import matplotlib libraries
from matplotlib import pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Landmark approach


In [ ]:
!pip install mediapipe

In [ ]:
!wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/float16/4?lite-format=tflite
input_size = 192
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()


In [ ]:
# 2. video
vidcap = cv2.VideoCapture('test2.mp4')
success,image = vidcap.read()
count = 0
#out2 = main_visualize(image)
while success:
  #out2 = tf.concat([out2, main_visualize(image)], axis=0)
  #cv2.imwrite("output/frame%d.jpg" % count, out)     # save frame as JPEG file      
  #success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

In [ ]:
import json
import os

# change Working directory to the video folder
os.chdir('Deepfake-Detection-Challenge/train_sample_videos')

In [ ]:
# load labels from json file
with open('metadata.json','r') as j:
     labels = json.loads(j.read())
labels

In [ ]:
count = 0
# loop throug all the files in a 
for video in os.listdir():
  if video.endswith('.mp4'):
    v = cv2.VideoCapture(video)
    imgs = video_to_images(v)
    o = extract_landmarks(imgs)
    # create first df 
    if count == 0:
      dfs = join_image_dfs(o)
      dfs['label'] = labels[video]['label']
      dfs['VideoId'] = video
      count += 1
    # append to previous df
    else:
      df = join_image_dfs(o)
      df['label'] = labels[video]['label']
      df['VideoId'] = video
      dfs = pd.concat([dfs, df])
      count += 1
  #if count > 5:
   # break
  print('done ' + str(count) )

In [ ]:
dfs.to_csv('out1.csv')
!cp out1.csv "/content/drive/My Drive/"

In [ ]:
dfs_nona = dfs.dropna()
dfs_nona = dfs_nona.reset_index(drop=True)
dfs_nona.to_csv('out_nona.csv')
!cp out_nona.csv "/content/drive/My Drive/"

In [ ]:
dfs_nona

##### Test

In [ ]:
v_fake = cv2.VideoCapture('etmcruaihe.mp4')
v_original = cv2.VideoCapture('afoovlsmtx.mp4')

imgs_fake = video_to_images(v_fake)
imgs_original = video_to_images(v_original)


In [ ]:
#output = draw_landmarks_on_img(imgs)
output_fake = extract_landmarks(imgs_fake)
output_original = extract_landmarks(imgs_original)


In [ ]:
video_df_fake = join_image_dfs(output_fake)
video_df_original = join_image_dfs(output_original)

In [ ]:
# compare fake and real
#video_df_original.compare(video_df_fake)
#video_df_fake

df_con = pd.concat([video_df_original, video_df_fake], keys=['original', 'fake'],names=['Series name', 'Row ID'])
df_con.groupby('Series name').mean()

,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,...,z_474,x_475,y_475,z_475,x_476,y_476,z_476,x_477,y_477,z_477
Series name,,,,,,,,,,,,,,,,,,,,,
original,0.401152,0.471183,-0.037070,0.402029,0.454824,-0.050685,0.403437,0.462373,-0.032038,0.393824,...,0.018203,0.448639,0.424146,0.018203,0.439120,0.428233,0.018203,0.448677,0.432417,0.018203
fake,0.402926,0.471003,-0.038333,0.399389,0.453024,-0.051811,0.401056,0.460871,-0.033252,0.391555,...,0.017947,0.448014,0.423959,0.017947,0.438819,0.428123,0.017947,0.447709,0.432381,0.017947


#### Face Forensic Dataset

In [ ]:
import os
os.chdir('/content/faceforensics/manipulated_sequences/Face2Face/c23/videos')


In [ ]:
count = 0
dfs_list_fake = []
# loop throug all the files in a 
for video in os.listdir():
  if video.endswith('.mp4'):
    v = cv2.VideoCapture(video)
    imgs = video_to_images(v)
    o = extract_landmarks(imgs)
    df = join_image_dfs(o)
    df['VideoId'] = video
    #dfs = pd.concat([dfs, df])
    dfs_list_fake.append(df)
    count += 1
  if count > 50:
    break
  print('done ' + str(count) )

https://www.tensorflow.org/guide/ragged_tensor

In [ ]:
dfs_transformed_list_fake = transform_dfs(dfs_list_fake)

# ragged tensro shape: (num_videos, num_frames, num_landmarks(478), num_cordinates(3))
array_fake = [x.to_numpy() for x in dfs_transformed_list_fake]
#data_fake = tf.ragged.constant(array_fake, dtype=float)
#data_fake.bounding_shape()

In [ ]:
#np.array(array_fake)
np.savez_compressed('/content/drive/MyDrive/data_face2face_50.npz', fake = np.array(array_fake))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [ ]:


#print(array_fake.shape)
np.load('/content/drive/MyDrive/data_face2face_100.npz', allow_pickle=True)['fake']

In [ ]:
import os
os.chdir('/content/faceforensics/original_sequences/youtube/c23/videos')

In [ ]:
count = 0
dfs_list_original = []
# loop throug all the files in a 
for video in os.listdir():
  if video.endswith('.mp4'):
    v = cv2.VideoCapture(video)
    imgs = video_to_images(v)
    o = extract_landmarks(imgs)
    df = join_image_dfs(o)
    df['VideoId'] = video
    #dfs = pd.concat([dfs, df])
    dfs_list_original.append(df)
    count += 1
  if count > 50:
    break
  print('done ' + str(count) )

In [ ]:
dfs_transformed_list_original = transform_dfs(dfs_list_original)
array_original = [x.to_numpy() for x in dfs_transformed_list_original]
# ragged tensor shape: (num_videos, num_frames, num_landmarks(478), num_cordinates(3))
#data_original = tf.ragged.constant([x.to_numpy() for x in dfs_transformed_list_original], dtype='float32' )
#data_original.bounding_shape()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [ ]:
np.savez_compressed('/content/drive/MyDrive/data_face2face_original_50.npz', original = np.array(array_original))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [ ]:
data_fake = tf.ragged.constant(np.load('/content/drive/MyDrive/data_face2face_50.npz', allow_pickle=True)['fake'], dtype='float32' )

In [ ]:
data_original = tf.ragged.constant(np.load('/content/drive/MyDrive/data_face2face_original_50.npz', allow_pickle=True)['original'], dtype='float32' )

In [ ]:
data = tf.concat([data_fake, data_original], axis = 0)
data_fake = []
data_original = []

In [ ]:
y = tf.constant([0]*51 + [1]*51, dtype='int32')

In [ ]:
data = data.to_sparse()
data = tf.sparse.to_dense(data)

In [ ]:
data.shape

TensorShape([102, 1497, 478, 3])

# Training

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, shuffle=True, random_state=42, test_size=0.2)

In [ ]:
tf.random.set_seed(42)

# ragged_input = tf.keras.layers.Input(shape=[], dtype=tf.float32, ragged=True)

model_4 = tf.keras.Sequential([
	tf.keras.layers.Input(shape=[1497, 478, 3], dtype='float32'),
	tf.keras.layers.Dense(10, activation = "relu"),
	tf.keras.layers.Dense(1)
])

model_4.compile(loss = "binary_crossentropy",
				optimizer = tf.keras.optimizers.Adam(),
				metrics = ["accuracy"])

history = model_4.fit(tf.sparse.to_dense(data_sparse), labels, epochs = 20)

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Activation, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential

tf.random.set_seed(42)

# Create a CNN model (same as Tiny VGG - https://poloclub.github.io/cnn-explainer/)
model_1 = Sequential([
  Input(shape=[1497, 478, 3], dtype=tf.float32, ragged=False),
  Conv2D(filters=10, 
         kernel_size=3, # can also be (3, 3)
         strides=1,
         padding="valid",
         activation="relu"),
         #input_shape=(224, 224, 3)), # first layer specifies input shape (height, width, colour channels)
  Conv2D(10, 3, activation="relu"),
  MaxPool2D(pool_size=2, # pool_size can also be (2, 2)
            padding="valid"), # padding can also be 'same'
  Conv2D(10, 3, activation="relu"),
  Conv2D(10, 3, activation="relu"), # activation='relu' == tf.keras.layers.Activations(tf.nn.relu)
  MaxPool2D(2),
  Flatten(),
  Dense(10, activation="relu"), # binary activation output
  Dense(100, activation="relu"),
  Dense(1, activation="sigmoid")
])

# Compile the model
model_1.compile(loss="binary_crossentropy",
              optimizer=Adam(),
              metrics=["accuracy"])

# Fit the model
history_1 = model_1.fit(data,
                        y,
                        epochs=5)

Epoch 1/5
7/7 [==============================] - 184s 25s/step - loss: nan - accuracy: 0.5050
Epoch 2/5
7/7 [==============================] - 170s 24s/step - loss: nan - accuracy: 0.5000
Epoch 3/5
7/7 [==============================] - 169s 24s/step - loss: nan - accuracy: 0.5000
Epoch 4/5
7/7 [==============================] - 175s 25s/step - loss: nan - accuracy: 0.5000
Epoch 5/5
7/7 [==============================] - 168s 24s/step - loss: nan - accuracy: 0.5000
